In [1]:
!pip install webdriver_manager


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install selenium


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import os
# os.getcwd()
# os.listdir('C:\\Users\\DSSAL\\Downloads')

In [4]:
# [fname for fname in os.listdir(os.getcwd()) if re.match('.+[.]zip', fname)]  # 'Files\(\d+\).?[.]DOCX'

In [5]:
# os.getcwd()

In [6]:

# -*- coding: utf-8 -*-
### PLEASE SCROLL ALL THE WAY DOWN FOR CRAWLING AND SCRAPING ACTIONS SCRIPT ###

# %% import libraries
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common import by
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support import expected_conditions
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

import time
import re
from tqdm import tqdm
import os
import shutil
import math
import sys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
#https://github.com/SergeyPirogov/webdriver_manager

# %% Setting all functions
def initiate_driver(browsertype):
    global driver
    global wait
    global actionChains

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    capabilities = {
    'browserName': 'chrome',
    'chromeOptions':  {
        'useAutomationExtension': False,
        'forceDevToolsScreenshot': True,
        'args': ['--start-maximized', '--disable-infobars']
    }
    } 
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_settings.popups" : 0,
        "profile.default_content_setting_values.automatic_downloads" : 1,
        "download.prompt_for_download": 'False',
        "download.default_directory" : download_path,
        "download.directory_upgrade": True,
        "safebrowsing.1enabled": True
        }
    chrome_options.add_experimental_option("prefs", prefs)


    if browsertype == 'chromium':
        driver = webdriver.Chrome(executable_path='/snap/bin/chromium.chromedriver', options=chrome_options)
    elif browsertype == 'chrome':
        #driver = webdriver.Chrome(executable_path='C:/Users/pmh/Desktop/chromedriver', options=chrome_options)
        driver = webdriver.Chrome(ChromeDriverManager().install())

    wait = WebDriverWait(driver, 300)
    actionChains = ActionChains(driver)
    driver.maximize_window()

def waitforspinner():
    wait.until(ec.invisibility_of_element_located((By.XPATH, '//div[@class="box spinner-interactive"]')))

def open_url(url):
    driver.get(url)

def click_link(navigate_links):
    for link_caption in navigate_links:
        print('Clicking ' + link_caption)
        linkje = wait.until(ec.element_to_be_clickable((By.XPATH, f'//a[@data-text="{link_caption}"]')))
        linkje.click()

def select_source(sources):
    #Click 'select sources to search' radio-button
    if media == [] or  media[0] == 'all':
        pass
    else:
        print('Selecting source button')
        wait.until(ec.element_to_be_clickable((By.XPATH, "//input[@name='allsources']")))
        driver.find_element_by_xpath("//input[@name='allsources'][@ng-reflect-value='false']").click()
        time.sleep(1)
        if driver.find_element_by_xpath("//input[@name='allsources'][@ng-reflect-value='false']").is_selected() == False:
            driver.find_element_by_xpath("//input[@name='allsources'][@ng-reflect-value='false']").click()
        for source in sources:
            print('Selecting ' + source)
            linkje = wait.until(ec.element_to_be_clickable((By.XPATH, f"//input[@aria-label='{source}']")))
            linkje.send_keys(Keys.SPACE)

def enter_query(keywords):
    print('Entering Query: ' + keywords)
    driver.find_element_by_xpath("//*[@id='searchTerms']").send_keys(keywords)
    driver.implicitly_wait(1)
    driver.find_element_by_xpath("//*[@id='searchTerms']").send_keys(Keys.ENTER)
    driver.implicitly_wait(2)
    print('Waiting for results to load')
#     wait.until(ec.visibility_of_element_located((By.XPATH, "//*[@class='pagination']")))

def filter_date(start_date, end_date):
    print('Filtering date from ' + start_date + ' to ' + end_date)
    ###날짜 지정 ###
    #Timeline 클릭
    tl_btn = '//button[@aria-label="Expand Timeline"]'
    wait.until(ec.presence_of_element_located((By.XPATH, tl_btn)))
    driver.execute_script("arguments[0].click();",driver.find_element_by_xpath(tl_btn))

    #사작년도 삽입
    time.sleep(1)
    wait.until(ec.visibility_of_element_located((By.XPATH, '//div[@class="datepicker-container"]')))
    st_date = driver.find_element_by_xpath('//*[@id="sidebar"]/div/ln-postfilters/div/div/timelinefilter/filterpod/div[2]/div/div[2]/div[2]/div[1]/div[1]/datepicker/div/input')
    st_date.clear()
    for x in range(10): #clear()와Keys.CONTRAL + "a"작동이 안됨. for문이용해 백스페이스로 지움
        st_date.send_keys(Keys.BACKSPACE)
    st_date.send_keys(start_date)

    #마지막년도 삽입
    ed_date = driver.find_element_by_xpath('//*[@id="sidebar"]/div/ln-postfilters/div/div/timelinefilter/filterpod/div[2]/div/div[2]/div[2]/div[1]/div[2]/datepicker/div/input')
    ed_date.clear()
    for x in range(10):
        ed_date.send_keys(Keys.BACKSPACE)
    ed_date.send_keys(end_date)
    ed_date.send_keys(Keys.TAB)

    #날짜설정 확인클릭
    dt_btn = driver.find_element_by_xpath('//*[@id="sidebar"]/div/ln-postfilters/div/div/timelinefilter/filterpod/div[2]/div/div[2]/div[2]/div[2]/button')
    driver.execute_script("arguments[0].click();",dt_btn)
    driver.implicitly_wait(3)

    waitforspinner()

def sort_results(ranking): #Check amount of search results and sort from new to old
    waitforspinner()
    wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="sortbymenulabel"]')))
    driver.find_element_by_xpath('//*[@id="sortbymenulabel"]').click()
    driver.implicitly_wait(1)

    editsort = wait.until(ec.element_to_be_clickable((By.XPATH, '//button[@data-action="editsort"]')))
    editsort.click()

    wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="mainSettings"]/div[2]/div/div[2]/div[1]/settingsgeneral/div/section[2]/select/option[3]')))
    driver.find_element_by_xpath('//*[@id="mainSettings"]/div[2]/div/div[2]/div[1]/settingsgeneral/div/section[2]/select/option[3]').click()
    driver.implicitly_wait(3)

    if ranking == 'newest':
        driver.find_element_by_xpath('//select[@id="sort-by-dropdown-2"]/option[text()="Date (newest-oldest)"]').click()
    elif ranking == 'oldest':
        driver.find_element_by_xpath('//select[@id="sort-by-dropdown-2"]/option[text()="Date (oldest-newest)"]').click()
    
    #Click save
    button_save = '//*[@id="saveChangesButton"]'
    wait.until(ec.element_to_be_clickable((By.XPATH, button_save)))
    driver.find_element_by_xpath(button_save).click()
    
    waitforspinner()

def count_results():
    
    label_results = driver.find_element_by_xpath('//*[@id="content"]/header/h2/span').text
    print('Output ' + label_results)
    
    resultcount = int(re.sub('[^0-9]+', '', label_results))
#     if resultcount > 9999:
#         sys.exit("More than 10.000 records. Please reduce timeline and try again...")

    global last_page 
    last_page = math.ceil(resultcount / 50)
    #  last_page = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/form/div[2]/nav/ol/li[6]/a').text)
    # 10000개 이상 데이터 수집시
    print(last_page)
    print('Found ' + str(resultcount) + ' articles on ' + str(last_page) + ' pages')

def filter_duplicates():
    print('Enabling duplicate filter')
    dropdown_duplicate = "//button[@class='current trigger collapsed icon la-TriangleDownAfter']"
    wait.until(ec.element_to_be_clickable((By.XPATH, dropdown_duplicate)))
    driver.find_element_by_xpath(dropdown_duplicate).click()
    driver.implicitly_wait(1)
    driver.find_element_by_xpath("//button[@data-value='moderate']").click()

    waitforspinner()

def selectallarticles():
    waitforspinner()
    wait.until(ec.visibility_of_element_located((By.XPATH, '//*[@data-action="selectall"]')))
    driver.find_element_by_xpath('//*[@data-action="selectall"]').send_keys(Keys.SPACE)
    time.sleep(1)
    if driver.find_element_by_xpath('//*[@data-action="selectall"]').is_selected() == False:
        driver.find_element_by_xpath('//*[@data-action="selectall"]').send_keys(Keys.SPACE)
    wait.until(ec.visibility_of_element_located((By.XPATH, '//*[@class="select more"]')))

def opendownloaddialog():
    wait.until(ec.visibility_of_element_located((By.XPATH, '//*[@data-action="downloadopt"]')))
    driver.find_element_by_xpath('//*[@data-action="downloadopt"]').send_keys(Keys.SPACE)

def setdoctype(doctype):
    wait.until(ec.visibility_of_element_located((By.XPATH, '//*[@data-action="download"]')))
    if doctype == 'docx':    
        driver.find_element_by_xpath('//*[@id="Docx"]').click() #select Docx format"
    elif doctype == 'rtf':
        driver.find_element_by_xpath('//*[@id="Rtf"]').click() #select Rtf format"
    elif doctype == 'pdf':
        driver.find_element_by_xpath('//*[@id="Pdf"]').click() #select Rtf format"

def clickdownload():
    wait.until(ec.visibility_of_element_located((By.XPATH, '//*[@data-action="download"]')))
    driver.find_element_by_css_selector('body > aside > footer > div > button.button.primary').send_keys(Keys.SPACE)        

def downloadf():
    selectallarticles()
    opendownloaddialog()
    global download_settings
    if download_settings is False:
        # Basic options
        # wait.until(ec.visibility_of_element_located((By.XPATH, '//*[@data-action="download"]')))
        # driver.find_element_by_xpath('//*[@id="Docx"]').click() #select Docx format"
        # #driver.find_element_by_xpath('//*[@id="Rtf"]').click() #select Rtf format"
        # driver.implicitly_wait(3)
        setdoctype(doctype)

        # Formatting options for LexisNexisTools
        driver.find_element_by_xpath('//*[@id="tab-FormattingOptions"]').click()
        time.sleep(1)
        # driver.find_element_by_xpath('//*[@id="IncludeCoverPage"]').click()
        driver.find_element_by_xpath('//*[@id="DisplayFirstLastNameEnabled"]').click()
        # driver.find_element_by_xpath('//*[@id="PageNumberSelected"]').click()
        driver.find_element_by_xpath('//*[@id="EmbeddedReferences"]').click()

        download_settings = True

    clickdownload()


    try:
        #Wait for the job description box to appear
        #wait.until(ec.visibility_of_element_located((By.XPATH, '//div[@class="job-description"]')))
        wait.until(ec.visibility_of_element_located((By.XPATH, '//span[@class="status-message success"]')))
        
        #Wait for job description dialog to disappear
        wait.until(ec.invisibility_of_element_located((By.XPATH, '//span[@class="status-message success"]')))
        #time.sleep(6)
    except:
        driver.refresh()
        selectallarticles()
        opendownloaddialog()
        clickdownload()

    i = 1
    while not len([fname for fname in os.listdir() if re.match('Files\(\d+\).?[.DOCX]', fname)])!=0: # download폴더에 code위치하기
        #print(i)
        i += 1
        time.sleep(0.5)
        if i > 50:
            print('oh oh not good')
            break 
    
    # 방금 다운 받은 파일 이름 바꾸기 -> 중복방지
#     filename = max([download_path + '/' + f for f in os.listdir(download_path)],key=os.path.getctime)
#     shutil.move(filename,os.path.join(download_path,f"{batch_name}_{num}.docx")) ## 파일 이름 + num 설정

def start_downloading():
    global num
    for num in tqdm(range(0, (last_page))):
        time.sleep(1)
        downloadf()
        try:
            driver.find_element_by_xpath("//a[@data-action='nextpage']").click()
            #time.sleep(5)
            waitforspinner()
        except:
            print('Seems like there are no more pages left')
            
def select_50data():
        ### 다운로드 수 50개 설정 ###
    driver.find_element_by_xpath('//*[@id="sortbymenulabel"]').click()
    driver.implicitly_wait(3)
    
    
    driver.find_element_by_xpath('//*[@id="results-list-delivery-toolbar"]/div/ul[2]/li/div/div/button[6]/span').click()
    driver.implicitly_wait(3)
    
    #driver_hl.find_element_by_xpath('//*[@id="mainSettings"]/div[2]/div/div[2]/div[1]/settingsgeneral/div/section[2]/select').click()
    driver.find_element_by_xpath('//*[@id="mainSettings"]/div[2]/div/div[2]/div[1]/settingsgeneral/div/section[2]/select/option[3]').click()
    driver.implicitly_wait(3)
    
    driver.find_element_by_xpath('//*[@id="saveChangesButton"]').click()
    #driver_hl.find_element_by_xpath('//*[@id="discardChangesButton"]').click()
    driver.implicitly_wait(3)
    
    ## sort : old -> new
    driver.find_element_by_xpath('//*[@id="sortbymenulabel"]').click()
    driver.find_element_by_xpath('//*[@id="results-list-delivery-toolbar"]/div/ul[2]/li/div/div/button[5]').click()
    
    ## 그룹 중복 설정 (Moderate)##
    #driver.find_element_by_xpath('//*[@id="content"]/header/div/div/button').click()
    #driver.find_element_by_xpath('//*[@id="content"]/header/div/div/aside/ul/li[3]/button').click()b
    

### navigate_link 중 publication종류: 두칸띄어쓰기 주의
### 브라우저에서 파일다운로드 경로 설정 후 downloadf() 코드 하단 print부분 수정 (docx 파일 찾도록)
### 키워드 리스트에 키워드 추가

In [18]:
# %% Action parameters ###### Start tweaking from here

## First page to open
start_page = "http://openlink.ca.skku.edu/link.n2s?url=http://advance.lexis.com/?identityprofileid=6QNHW655035"

## Navigation path based on link's 'aria-label' attribute
#navigate_links = ['News', 'Major World  Newspapers']
navigate_links = ['News', 'Major World  Newspapers' ] # 두칸 띄어쓰기? ;; 극혐


## Select sources (leave empty list or write 'all' to keep 'all sources')
media = ['all']
# c = [
#     'Metro (UK)' ,
#  'The Daily Mail and Mail on Sunday (London)',
#  'The Sun (England)',
#  'The Mirror (The Daily Mirror and The Sunday Mirror)',
#  'The Guardian (London)',
#  'The Times (London)'
#  ] #Only needed when selecting custom sources (by aria-label attribute in HTML)

## Query (keywords) to type in the search field
keywords_li = ["british museum","British museum",'Artifact','artifact']
keywords = ' OR '.join(keywords_li)

## Set date for filter 
start_date = "06/13/2022"
end_date = "07/01/2022"

## Download settings
batch_name = 'TESTING' #First part of filenames
download_path = '‪‪C:\\Users\\DSSAL\\Downloads' #Abosulte path to download directory
doctype = 'docx' #choose docx, pdf or rtf
download_settings = False #only change if not using LexisNexisTools



#%% The actual process
### EASY ADD OR REMOVE STEPS

initiate_driver('chrome') #Type either chrome or chromium depending on webdriver
open_url(start_page) #Open start page as set in the search parameters
click_link(navigate_links) #Click links according to navigation path in search parameters
select_source(media) #Select source selector radio button and select media by 'aria-label' stated in Search Parameters
enter_query(keywords) #Enter search query as stated in search parameters
filter_date(start_date, end_date) #Enable timeline filter by dates in search parameters
print(2)
#filter_duplicates() #Set duplicate filter to moderate
#sort_results('newest') #Show newest results on top
select_50data() # Select 50 data
count_results() #REQUIRED count results and calculate pages
start_downloading() #There we goooooooo

driver.quit()
print("\n That's it bro")


# %% DUTCH NEWS EXAMPLE
# ### DUTCH NEWS EXAMPLE


# ## Navigation path based on link's 'aria-label' attribute
# navigate_links = ['News', 'International By Country', 'Netherlands' ]


# ## Select sources (leave empty list or write 'all' to keep 'all sources')
# media = [
#     'Algemeen Nederlands Persbureau ANP',
#     'AD/Algemeen Dagblad',
#     'AD/Algemeen Dagblad.nl',
#     'De Telegraaf',
#     'De Telegraaf.nl',
#     'Metro (NL)',
#     'de Volkskrant',
#     'De Volkskrant.nl'
# ]

# ## Query (keywords) to type in the search field
# keywords = 'thuiswerken'

# ## Set date for filter 
# start_date = "01/01/2020"
# end_date = "12/31/2020"

# ## Download settings
# batch_name = 'NLNEWS2020' #First part of filenames
# download_path = '/home/pim/newscollect/NLNEWS' #Abosulte path to download directory
# doctype = 'docx' #choose docx, pdf or rtf
# download_settings = False #only change if not using LexisNexisTools

# ## The actual scraping script
# initiate_driver('chromium') 
# open_url(start_page)
# click_link(navigate_links) 
# select_source(media) 
# enter_query(keywords) 
# filter_date(start_date, end_date) 
# filter_duplicates() 
# sort_results('newest') 
# count_results() 
# start_downloading()

# driver.quit()
# print("\n That's it bro")

# %%

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST driver version for 104.0.5112


[WDM] - Driver [C:\Users\DSSAL\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Clicking News
Clicking Major World  Newspapers
Entering Query: british museum OR British museum OR Artifact OR artifact
Waiting for results to load
Filtering date from 06/13/2022 to 07/01/2022
2
Output News (177)
4
Found 177 articles on 4 pages


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:59<00:00, 29.83s/it]

Seems like there are no more pages left



 That's it bro


In [19]:
driver.quit()
driver.close()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57589): Max retries exceeded with url: /session/07635b2513cb11368e27d154bab9af7c/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002CA2745C088>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))